## Simulation Framework 

In [14]:
import import_ipynb
import math 
import importlib
import random

# import functions from model_evaluation.ipynb  
import model_evaluation 

In [56]:
# simulation parameters 

# number of groups
n_groups = 4 

# number of agents 
n_agents = 4

In [57]:
# create group distributions 
def distribute_agents(n_agents, n_groups):
    """
    This function randomly selects a subset of group indices for each agent. The number of groups for each agent is randomly chosen between 1 and the total number of groups.
    :returns (list of lists): Each inner list represents the agents assigned to a particular group.
    """
    group_dist = [[] for _ in range(n_groups)]

    for agent in range(n_agents):
        groups = random.sample(range(n_groups), random.randint(1, n_groups))
        for group in groups:
            group_dist[group].append(agent)

    return group_dist

In [68]:
def gradual_distribution_transition(n_agents, n_groups, iterations):
    """
    This function starts with a distribution where each group contains only one agent and gradually modifies it in each iteration.
    :returns (list of lists): Each inner list represents the agents assigned to a particular group.
    """
    # Initial distribution: Each group contains only one agent
    group_dist = [[agent] for agent in range(n_agents)]

    for iteration in range(iterations):
        # Randomly select a group
        source_group = random.randint(0, n_groups - 1)
        destination_group = random.randint(0, n_groups - 1)

        # Randomly select an agent from the source group
        if group_dist[source_group]:
            agent_to_move = random.choice(group_dist[source_group])

            # Move the agent to the destination group if not already present
            if agent_to_move not in group_dist[destination_group]:
                group_dist[destination_group].append(agent_to_move)

        print(f"Iteration {iteration + 1}: {group_dist}")

    return group_dist

# Example usage
n_agents = 4
n_groups = 4
iterations = 100  # Adjust the number of iterations as needed
result = gradual_distribution_transition(n_agents, n_groups, iterations)


Iteration 1: [[0], [1], [2], [3]]
Iteration 2: [[0], [1], [2], [3]]
Iteration 3: [[0], [1], [2], [3, 0]]
Iteration 4: [[0], [1], [2, 1], [3, 0]]
Iteration 5: [[0], [1], [2, 1], [3, 0]]
Iteration 6: [[0], [1], [2, 1], [3, 0, 1]]
Iteration 7: [[0], [1, 0], [2, 1], [3, 0, 1]]
Iteration 8: [[0], [1, 0], [2, 1], [3, 0, 1]]
Iteration 9: [[0, 1], [1, 0], [2, 1], [3, 0, 1]]
Iteration 10: [[0, 1], [1, 0], [2, 1], [3, 0, 1]]
Iteration 11: [[0, 1], [1, 0], [2, 1], [3, 0, 1]]
Iteration 12: [[0, 1], [1, 0], [2, 1], [3, 0, 1, 2]]
Iteration 13: [[0, 1], [1, 0], [2, 1, 0], [3, 0, 1, 2]]
Iteration 14: [[0, 1], [1, 0], [2, 1, 0], [3, 0, 1, 2]]
Iteration 15: [[0, 1], [1, 0], [2, 1, 0], [3, 0, 1, 2]]
Iteration 16: [[0, 1], [1, 0], [2, 1, 0], [3, 0, 1, 2]]
Iteration 17: [[0, 1], [1, 0, 2], [2, 1, 0], [3, 0, 1, 2]]
Iteration 18: [[0, 1], [1, 0, 2], [2, 1, 0], [3, 0, 1, 2]]
Iteration 19: [[0, 1], [1, 0, 2], [2, 1, 0], [3, 0, 1, 2]]
Iteration 20: [[0, 1], [1, 0, 2], [2, 1, 0], [3, 0, 1, 2]]
Iteration 21: [[0,

In [65]:
group_dist = distribute_agents(n_agents=n_agents, n_groups=n_groups)
print(group_dist)

[[2, 3], [0, 2], [2], [1, 2, 3]]


In [59]:
# define vote distribution 
def vote_distribution_fixed(n_agents, votes_agent):
    return [votes_agent] * n_agents

In [60]:
vote_dist = vote_distribution_fixed(n_agents=n_agents, votes_agent=5)
print(vote_dist)

[5, 5, 5, 5]


In [2]:
# Define input variables  
group_dist = [[0], [1], [2], [3]] 
vote_dist = [5, 5, 5, 5]

In [61]:
# Assuming this is the correct function signature
result_group_memberships = model_evaluation.group_memberships(group_dist, vote_dist)
print(result_group_memberships)

[[0, 2], [1], [1], [3]]


In [50]:
result_groups, result_aggregate_weight = model_evaluation.first_term(group_dist, vote_dist, result_group_memberships)
print(result_groups)
print(result_aggregate_weight)

{'group_0': {'agent_2': {'votes': 5, 'num_groups': 5, 'vote_weight': 1.0}, 'agent_4': {'votes': 5, 'num_groups': 5, 'vote_weight': 1.0}, 'agent_6': {'votes': 5, 'num_groups': 5, 'vote_weight': 1.0}, 'agent_8': {'votes': 5, 'num_groups': 4, 'vote_weight': 1.25}}, 'group_1': {'agent_0': {'votes': 5, 'num_groups': 3, 'vote_weight': 1.6666666666666667}, 'agent_2': {'votes': 5, 'num_groups': 5, 'vote_weight': 1.0}, 'agent_3': {'votes': 5, 'num_groups': 4, 'vote_weight': 1.25}, 'agent_4': {'votes': 5, 'num_groups': 5, 'vote_weight': 1.0}, 'agent_6': {'votes': 5, 'num_groups': 5, 'vote_weight': 1.0}, 'agent_7': {'votes': 5, 'num_groups': 2, 'vote_weight': 2.5}, 'agent_8': {'votes': 5, 'num_groups': 4, 'vote_weight': 1.25}}, 'group_2': {'agent_1': {'votes': 5, 'num_groups': 3, 'vote_weight': 1.6666666666666667}, 'agent_2': {'votes': 5, 'num_groups': 5, 'vote_weight': 1.0}, 'agent_3': {'votes': 5, 'num_groups': 4, 'vote_weight': 1.25}, 'agent_4': {'votes': 5, 'num_groups': 5, 'vote_weight': 1.0

In [51]:
result_individual, result_aggregated = model_evaluation.interaction_terms(group_dist, result_group_memberships, vote_dist)
print(result_individual)
print(result_aggregated)

{'group_0': {'other_group_1': {'components_int_term1': {'agent_i_2': {'vote_attenuation_i': 2.23606797749979, 'num_groups_i': 5, 'vote_weight_i': 0.447213595499958}, 'agent_i_4': {'vote_attenuation_i': 2.23606797749979, 'num_groups_i': 5, 'vote_weight_i': 0.447213595499958}, 'agent_i_6': {'vote_attenuation_i': 2.23606797749979, 'num_groups_i': 5, 'vote_weight_i': 0.447213595499958}, 'agent_i_8': {'vote_attenuation_i': 2.23606797749979, 'num_groups_i': 4, 'vote_weight_i': 0.5590169943749475}}, 'components_int_term2': {'agent_j_0': {'vote_attenuation_j': 2.23606797749979, 'num_groups_j': 3, 'vote_weight_j': 0.7453559924999299}, 'agent_j_2': {'vote_attenuation_j': 2.23606797749979, 'num_groups_j': 5, 'vote_weight_j': 0.447213595499958}, 'agent_j_3': {'vote_attenuation_j': 2.23606797749979, 'num_groups_j': 4, 'vote_weight_j': 0.5590169943749475}, 'agent_j_4': {'vote_attenuation_j': 2.23606797749979, 'num_groups_j': 5, 'vote_weight_j': 0.447213595499958}, 'agent_j_6': {'vote_attenuation_j':

In [53]:
plurality_score = model_evaluation.connection_oriented_cluster_match(result_aggregate_weight, result_aggregated)
print(plurality_score)

11.647739543735016


In [54]:
result_quadratic_voting, result_sum_quadratic_votes = model_evaluation.quadratic_voting(vote_dist)
print(result_quadratic_voting)
print(result_sum_quadratic_votes)

{0: 2.23606797749979, 1: 2.23606797749979, 2: 2.23606797749979, 3: 2.23606797749979, 4: 2.23606797749979, 5: 2.23606797749979, 6: 2.23606797749979, 7: 2.23606797749979, 8: 2.23606797749979, 9: 2.23606797749979}
22.360679774997898


In [55]:
# Jaccard Index 
pairwise_jaccard_matrix = model_evaluation.pairwise_jaccard_similarity(group_dist)
print("Pairwise Jaccard Similarity Matrix:")
for row in pairwise_jaccard_matrix:
    print(row)

global_similarity = model_evaluation.global_jaccard_similarity(pairwise_jaccard_matrix)
print("Global Jaccard Similarity:", global_similarity)

Pairwise Jaccard Similarity Matrix:
[0.0, 0.5714285714285714, 0.5, 0.5, 0.375]
[0.5714285714285714, 0.0, 0.5, 0.875, 0.5555555555555556]
[0.5, 0.5, 0.0, 0.6, 0.6666666666666666]
[0.5, 0.875, 0.6, 0.0, 0.6666666666666666]
[0.375, 0.5555555555555556, 0.6666666666666666, 0.6666666666666666, 0.0]
Global Jaccard Similarity: 0.581031746031746
